In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pickle

In [3]:
datapath = "../input/danger-of-extinction-animal-image-set/Danger Of Extinction"
labels = {}
enum = 0
for item in sorted(os.listdir(datapath)):
    labels[enum] = item
    enum+=1
labels

In [4]:
DataGen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.1
)
train_datagen = DataGen.flow_from_directory(
    datapath,
    batch_size = 32,
    target_size = (299,299),
    class_mode = "categorical",
    subset = "training")
validation_datagen = DataGen.flow_from_directory(
    datapath,
    batch_size = 32,
    target_size = (299,299),
    class_mode = "categorical",
    subset = "validation")

In [5]:
print(train_datagen.class_indices)
print(validation_datagen.class_indices)

In [6]:
from tensorflow.keras.applications import InceptionResNetV2

inceptionresnet = InceptionResNetV2(include_top=False,
    weights="imagenet",
    input_shape=(299,299,3) 
)

In [7]:
for layer in inceptionresnet.layers:
    layer.trainable = False
base_output = GlobalAveragePooling2D()(inceptionresnet.output)
predictions = Dense(11, activation = "softmax", name = "predictions")(base_output)
my_inceptionresnet_model = Model(inputs = inceptionresnet.input, outputs = predictions)

In [8]:
early_stopping = EarlyStopping(monitor = "val_loss", patience=5)
model_save = ModelCheckpoint("inceptionresnet_model.h5", save_best_only=True, verbose=1)

my_inceptionresnet_model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
history_inceptionresnet = my_inceptionresnet_model.fit(train_datagen, validation_data=validation_datagen, epochs=50, 
             steps_per_epoch = train_datagen.samples//32,
             callbacks=[early_stopping, model_save])

In [9]:
with open('./trainHistoryInceptionResNet', 'wb') as file_pi:
    pickle.dump(history_inceptionresnet.history, file_pi)